In [358]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, argrelmin
import pandas as pd
from collections import Counter
import os

from pathlib import Path

from scipy import signal
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [359]:
auscultatory_sbp = []
auscultatory_pressure = []

In [360]:
path_aurora_dataset = Path('/Users/cslinxs/Desktop/aurora_dataset/aurora_dataset/')
auscultatory_data = path_aurora_dataset / "measurements_auscultatory.tsv"
basic_mesures_data = pd.read_csv(auscultatory_data,sep = '\t').dropna(subset=["waveform_file_path"]).dropna(subset=["sbp"]).dropna(subset=["dbp"]) 
def get_stats(check_list):
    standard = float(np.std(check_list))
    mean = float(np.mean(check_list))
    return [min(check_list),mean-standard-standard,mean-standard,mean,mean + standard,mean +standard+standard ,max(check_list) ]


def get_50_avg_coordinates(path):
    
    df = pd.read_csv(path,sep = '\t')   
    pos_pressure = df["pressure"]

    neglist = [ -x for x in pos_pressure]

    # ADD THE DROPPING DWOAN AND GETMIN BELLOW a LINE
    min_peaks, _ = find_peaks(neglist, prominence=10,width=80 )


    # juts deleted stuff from both lists if its in tht 40 range 
    # THIS IS THE MOST IMPORTANT 
    most_accurate_mins = min_peaks

    diffrences_new_min = np.diff(most_accurate_mins)
    mean_new_min = float(diffrences_new_min.mean())

    overlapped_means = pd.DataFrame( )
    nans_list = np.full(int(mean_new_min- get_stats(diffrences_new_min)[1]), np.nan)
    
    for i in range(len(most_accurate_mins)-1):
        
        current_pulse_width = int(most_accurate_mins[i+1]) -int(most_accurate_mins[i])
        if (current_pulse_width)< get_stats(diffrences_new_min)[4] and (current_pulse_width)> get_stats(diffrences_new_min)[1]:
            y_range  = list(range(int(most_accurate_mins[i]),int(most_accurate_mins[i+1])))
            y_of_shift = [int(y - min(pos_pressure[y_range])) for y in pos_pressure[y_range]]


            start_end_line = np.linspace(y_of_shift[0], y_of_shift[-1], len(y_of_shift), endpoint=True)
            y_rotated = [xi - yi for xi, yi in zip(y_of_shift, start_end_line)]
            # print("roatte leny ",len(y_rotated))
            # print("added len " , len(list(y_rotated)[:int(mean_new_min)]), len(list(nans_list[:int(mean_new_min) - len(y_rotated)])))
            # print(int(mean_new_min) - len(y_rotated))
            
            if len(y_rotated)>=int(mean_new_min):
                overlapped_means[f"pulse {i}"] = list(y_rotated)[:int(mean_new_min)]
            else:
                overlapped_means[f"pulse {i}"] = list(y_rotated)[:int(mean_new_min)] + list(nans_list[:int(mean_new_min) - len(y_rotated)])

    # WORKS STAY LIKE TAHTA^^^^  MAKES THE MEAN THE HIGHEST VLAUE , 
    #maybe later make it wider but for now it would work the same!
    overlapped_means['mean'] = overlapped_means.mean(axis=1)
    all_overlaps_means = (overlapped_means.mean(axis=1))
    if np.isnan(all_overlaps_means):
        print("IS a bad file "+path)
    
    


    #resampled_y = signal.resample(all_overlaps_means, 50)
    #return resampled_y


def get_person_activity_data(person_id):
    filler_to_person = basic_mesures_data[basic_mesures_data["pid"] == (person_id)]
    

    for each_path in filler_to_person["waveform_file_path"]:
        auscultatory_pressure.append(get_50_avg_coordinates(path_aurora_dataset/each_path))


    return  filler_to_person["sbp"]


In [388]:
# this will be main loop ---- later....


for person in os.listdir(path_aurora_dataset/"measurements_auscultatory")[:100]:
    if "a" in person:
        auscultatory_sbp.extend(get_person_activity_data(person))

In [387]:
all_points = np.array(auscultatory_pressure)
all_points = all_points.reshape(all_points.shape[0], -1)


X_train, X_test, Y_train, Y_test = train_test_split(all_points, auscultatory_sbp, test_size=0.3)
model = LinearRegression()


the_fit= model.fit(all_points, auscultatory_sbp)
score  = model.score(all_points, auscultatory_sbp)
y_predict = model.predict(X_test)


for i in range(len(y_predict)):

    plt.vlines(x = Y_test[i], ymin =y_predict[i] , ymax = Y_test[i])


rms = mean_squared_error(Y_test, Y_test, squared=False)
print("mean Sq",mean_squared_error(Y_test, y_predict))
print("rms",mean_squared_error(Y_test, y_predict))
print("r2 ",r2_score(Y_test, y_predict))



plt.scatter(Y_test, y_predict,  color="black", )
plt.plot(Y_test, Y_test, color="blue", linewidth=2)

plt.title(f"tested on {len(Y_test)} graphs")

plt.xlabel("tested sbp")
plt.ylabel("predicted sbp")



ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
print(X_train.shape)

(95, 50)


In [ ]:
# # just testing
# path = "/Users/cslinxs/Desktop/aurora_dataset/aurora_dataset/measurements_auscultatory/a000/a000.initial.Calibration_start_1.tsv"
# print(get_50_avg_coordinates(path))

# plt.plot(get_50_avg_coordinates(path) )

# plt.title("average based on all rates with pandas")
# plt.ylabel("Pressure")
# plt.xlabel("time seconds")

# plt.show()
# #422.5

In [ ]:
# person_id ="a000"
# activity= "a000.initial.Calibration_start_1.tsv"
# auscultatory_data = "/Users/cslinxs/Desktop/all_weeks/aurorabp-sample-data/sample/measurements_auscultatory.tsv"
# basic_mesures_data = pd.read_csv(auscultatory_data,sep = '\t') 

# basic_mesures_data.dropna(subset="waveform_file_path", inplace=True)

# filter_to_person = basic_mesures_data[basic_mesures_data["pid"] == person_id]

# activity_file_path = f"measurements_auscultatory/{person_id}/{activity}"



# # #return filler_to_person[(filler_to_person['waveform_file_path'].str.contains(activity_file_path))]
# filter_to_activity = filter_to_person.loc[filter_to_person["waveform_file_path"].str.contains(activity_file_path)]


In [365]:

print(auscultatory_pressure)

[array([  0.36089917,   2.87360329,  20.57193124,  49.5321252 ,
        75.37280741,  94.11277016, 106.61052968, 114.02483407,
       117.49074848, 118.32607642, 117.25409039, 115.23215282,
       112.63922668, 109.75237553, 106.48967948, 102.84683151,
        98.61741327,  93.50118424,  86.74246788,  78.56971769,
        69.99599728,  62.77794475,  57.66835724,  54.88294437,
        53.90653978,  53.9730365 ,  54.117526  ,  53.77086269,
        52.91101203,  51.29802174,  49.00229806,  46.27807326,
        43.30876514,  40.28188635,  37.27305689,  34.33590792,
        31.57183042,  28.98242934,  26.59181854,  24.4500113 ,
        22.35392738,  20.19441863,  17.93334131,  15.55811962,
        12.9330208 ,  10.188056  ,   7.35617439,   5.19526412,
         3.56078722,   1.91129791]), array([  0.19598754,   2.82787613,  20.35933985,  50.40449776,
        77.68884519,  97.41764339, 110.53693966, 118.18647207,
       121.78850241, 122.79099293, 122.13325239, 120.49383419,
       118.303683

In [372]:
basic_mesures_data = pd.read_csv(auscultatory_data,sep = '\t').dropna(subset=["waveform_file_path"]).dropna(subset=["sbp"]).dropna(subset=["dbp"]) 

In [373]:
basic_mesures_data

,pid,phase,measurement,date_time,sbp,dbp,duration,pressure_quality,optical_quality,waveform_file_path,waveforms_generated,primary_systolic,primary_diastolic,secondary_systolic,secondary_diastolic,consensus_systolic_error,consensus_diastolic_error
0,a000,initial,Calibration start 1,2018-01-01 13:14:20,110.0,74.0,18.5,1.000,0.983,measurements_auscultatory/a000/a000.initial.Ca...,1,108.0,72.0,112.0,76.0,4.0,4.0
1,a000,initial,Calibration start 2,2018-01-01 13:16:15,113.0,73.0,17.8,1.000,0.950,measurements_auscultatory/a000/a000.initial.Ca...,1,114.0,70.0,112.0,76.0,2.0,6.0
3,a000,initial,Static challenge start 1,2018-01-01 13:19:49,121.0,71.0,21.0,1.000,0.928,measurements_auscultatory/a000/a000.initial.St...,1,120.0,72.0,122.0,70.0,2.0,2.0
4,a000,initial,Static challenge start 2,2018-01-01 13:21:44,111.0,73.0,20.2,1.000,1.000,measurements_auscultatory/a000/a000.initial.St...,1,110.0,72.0,112.0,74.0,2.0,2.0
5,a000,initial,Static challenge start 3,2018-01-01 13:23:46,108.0,73.0,18.8,0.000,0.954,measurements_auscultatory/a000/a000.initial.St...,1,108.0,74.0,108.0,72.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11592,a672,return,Temporal challenge start 2,2018-01-02 15:55:35,117.0,65.0,19.0,1.000,0.994,measurements_auscultatory/a672/a672.return.Tem...,1,117.0,65.0,NaN,NaN,0.0,0.0
11593,a672,return,Temporal challenge start 3,2018-01-02 15:57:27,115.0,64.0,18.2,1.000,0.970,measurements_auscultatory/a672/a672.return.Tem...,1,115.0,64.0,NaN,NaN,0.0,0.0
11594,a672,return,Temporal seated challenge 1,2018-01-02 16:03:02,110.0,68.0,18.3,0.955,0.890,measurements_auscultatory/a672/a672.return.Tem...,1,110.0,68.0,NaN,NaN,0.0,0.0
11595,a672,return,Temporal seated challenge 2,2018-01-02 16:04:45,114.0,59.0,25.7,1.000,0.900,measurements_auscultatory/a672/a672.return.Tem...,1,114.0,59.0,NaN,NaN,0.0,0.0


In [374]:
# 11427 - > 11418 - > 11417

SyntaxError: invalid syntax (3731497640.py, line 1)

In [376]:
print(len(auscultatory_pressure), len(auscultatory_sbp))

1707 1707


In [384]:
auscultatory_pressure.count(100)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [381]:
type(auscultatory_pressure)

list

In [385]:
print(auscultatory_pressure)

[array([  0.36089917,   2.87360329,  20.57193124,  49.5321252 ,
        75.37280741,  94.11277016, 106.61052968, 114.02483407,
       117.49074848, 118.32607642, 117.25409039, 115.23215282,
       112.63922668, 109.75237553, 106.48967948, 102.84683151,
        98.61741327,  93.50118424,  86.74246788,  78.56971769,
        69.99599728,  62.77794475,  57.66835724,  54.88294437,
        53.90653978,  53.9730365 ,  54.117526  ,  53.77086269,
        52.91101203,  51.29802174,  49.00229806,  46.27807326,
        43.30876514,  40.28188635,  37.27305689,  34.33590792,
        31.57183042,  28.98242934,  26.59181854,  24.4500113 ,
        22.35392738,  20.19441863,  17.93334131,  15.55811962,
        12.9330208 ,  10.188056  ,   7.35617439,   5.19526412,
         3.56078722,   1.91129791]), array([  0.19598754,   2.82787613,  20.35933985,  50.40449776,
        77.68884519,  97.41764339, 110.53693966, 118.18647207,
       121.78850241, 122.79099293, 122.13325239, 120.49383419,
       118.303683

In [386]:
print(auscultatory_sbp)

[120.0, 130.0, 135.5, 138.0, 142.5, 141.5, 142.0, 151.5, 151.0, 143.5, 152.0, 199.5, 121.5, 125.5, 129.0, 132.0, 135.5, 138.0, 138.0, 144.0, 144.0, 144.0, 149.0, 152.0, 146.0, 149.0, 144.0, 142.0, 143.0, 200.0, 224.0, 150.0, 155.0, 158.0, 154.0, 147.0, 148.0, 117.5, 118.5, 117.0, 124.0, 119.5, 116.0, 122.5, 116.5, 112.0, 119.0, 117.0, 121.5, 147.0, 126.0, 124.0, 122.5, 123.0, 120.0, 118.5, 116.5, 147.5, 149.0, 149.5, 153.0, 159.0, 157.5, 159.5, 154.5, 152.0, 151.5, 157.5, 160.5, 197.0, 206.0, 151.0, 154.5, 157.0, 170.5, 152.5, 165.5, 144.0, 147.0, 147.0, 152.0, 159.5, 155.0, 146.5, 142.0, 152.0, 191.0, 181.0, 163.0, 161.0, 165.0, 125.0, 124.0, 121.0, 126.0, 129.0, 126.0, 119.0, 122.0, 122.0, 122.0, 120.0, 125.0, 156.0, 154.0, 116.0, 117.0, 115.0, 110.0, 114.0, 118.0, 154.0, 152.0, 163.0, 157.5, 152.0, 149.0, 152.0, 156.5, 156.5, 158.0, 155.0, 156.0, 184.5, 193.0, 168.0, 163.5, 163.0, 163.5, 162.0, 162.0, 123.0, 122.5, 131.0, 121.5, 123.0, 121.5, 125.5, 118.5, 119.5, 118.0, 126.5, 132.5